## Code for web-scraping without using pandas read_html

### Import libraries

In [1]:
import requests
import lxml.html as lh
import numpy as np
import pandas as pd

### Choose number of pages and determine header

In [2]:
# choose number of pages of results we want to include from the IAAF lists (for both genders)
npage=15

# determine header to open the webpage (Inspect Element, Network, Headers)
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:67.0) Gecko/20100101 Firefox/67.0'}

### Web-scraping

In [3]:
# create empty final lists
tr1_elements_final = []
tr2_elements_final = []

# men's results
ip=1
while ip<=npage:
    
    # get url and create a page
    url1='http://www.iaaf.org/records/all-time-toplists/sprints/100-metres/outdoor/men/senior?page=' + str(ip)
    url2='http://www.iaaf.org/records/all-time-toplists/sprints/200-metres/outdoor/men/senior?page=' + str(ip)
    page1 = requests.get(url1, headers=headers)
    page2 = requests.get(url2, headers=headers)
    
    # store the contents of the website
    doc1 = lh.fromstring(page1.content)
    doc2 = lh.fromstring(page2.content)
    
    # parse data that are stored between <tr>..</tr> of HTML
    tr1_elements = doc1.xpath('//tr')
    tr2_elements = doc2.xpath('//tr')
    
    if ip==1:
        # in our case the table starts at position 18 (with the header)
        tr1_elements_final += tr1_elements[18:]
        tr2_elements_final += tr2_elements[18:]
    else:
        # without including the header again
        tr1_elements_final += tr1_elements[19:]
        tr2_elements_final += tr2_elements[19:]
        
    ip+=1
    
# women's results
ip=1
while ip<=npage:
    url1='http://www.iaaf.org/records/all-time-toplists/sprints/100-metres/outdoor/women/senior?page=' + str(ip)
    url2='http://www.iaaf.org/records/all-time-toplists/sprints/200-metres/outdoor/women/senior?page=' + str(ip)
    page1 = requests.get(url1, headers=headers)
    page2 = requests.get(url2, headers=headers)
    doc1 = lh.fromstring(page1.content)
    doc2 = lh.fromstring(page2.content)
    tr1_elements = doc1.xpath('//tr')
    tr2_elements = doc2.xpath('//tr')
    tr1_elements_final += tr1_elements[19:]
    tr2_elements_final += tr2_elements[19:]
    ip+=1

In [4]:
# create empty list
col1=[]
col2=[]

#For each row, store each first element (header) and an empty list
i=0
for t in tr1_elements_final[0]:
    i+=1
    name=t.text_content()
    name=name.strip()
    #print ('%d:"%s"'%(i,name))
    col1.append((name,[]))

i=0
for t in tr2_elements_final[0]:
    i+=1
    name=t.text_content()
    name=name.strip()
    #print ('%d:"%s"'%(i,name))
    col2.append((name,[]))

In [5]:
#Since our first row is the header, data is stored on the second row onwards
for j in range(1,len(tr1_elements_final)):
    #T is our j'th row
    T=tr1_elements_final[j]
    
    #If row is not of size 11, the //tr data is not from our table 
    if len(T)!=11:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content()
        data=data.strip()
        #Append the data to the empty list of the i'th column
        col1[i][1].append(data)
        #Increment i for the next column
        i+=1
        
for j in range(1,len(tr2_elements_final)):
    #T is our j'th row
    T=tr2_elements_final[j]
    
    #If row is not of size 11, the //tr data is not from our table 
    if len(T)!=11:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content()
        data=data.strip()
        #Append the data to the empty list of the i'th column
        col2[i][1].append(data)
        #Increment i for the next column
        i+=1

### Create pandas DataFrames

In [6]:
Dict1={title:column for (title,column) in col1}
Dict2={title:column for (title,column) in col2}

# there's a key with all blank entries so we get rid of it
del Dict1['']
del Dict2['']

# create DataFrames
df1=pd.DataFrame(Dict1)
df2=pd.DataFrame(Dict2)

# changing header name
df1.rename(columns={'Mark':'100m'}, inplace=True)
df2.rename(columns={'Mark':'200m'}, inplace=True)

In [7]:
df1.head()

,Rank,100m,WIND,Competitor,DOB,Nat,Pos,Venue,Date,Results Score
0,1,9.58,+0.9,Usain BOLT,21 AUG 1986,JAM,1,"Olympiastadion, Berlin (GER)",16 AUG 2009,1356
1,2,9.69,+2.0,Tyson GAY,09 AUG 1982,USA,1,Shanghai (CHN),20 SEP 2009,1316
2,2,9.69,-0.1,Yohan BLAKE,26 DEC 1989,JAM,1,"Pontaise, Lausanne (SUI)",23 AUG 2012,1316
3,4,9.72,+0.2,Asafa POWELL,23 NOV 1982,JAM,1f1,"Pontaise, Lausanne (SUI)",02 SEP 2008,1305
4,5,9.74,+0.9,Justin GATLIN,10 FEB 1982,USA,1,"Hamad Bin Suhaim, Doha (QAT)",15 MAY 2015,1298


In [9]:
df2.tail()

,Rank,200m,WIND,Competitor,DOB,Nat,Pos,Venue,Date,Results Score
2995,1474,23.35,+0.4,Mayumi WATANABE,06 JUN 1983,JPN,1f1,Fukuroi (JPN),03 MAY 2013,1099
2996,1474,23.35,+2.0,Jasmine WEBB,13 FEB 1992,USA,1,"Lake Charles, LA (USA)",12 MAY 2013,1099
2997,1474,23.35,+1.1,Alyona MAMINA,30 MAY 1990,RUS,3,Zhukovskiy (RUS),30 JUN 2013,1099
2998,1474,23.35,+0.9,Huijun LIN,01 FEB 1993,CHN,1h3,Incheon (KOR),30 SEP 2014,1099
2999,1474,23.35,+1.6,Dominique KIMPEL,06 FEB 1993,USA,5f4,"Gainesville, FL (USA)",03 APR 2015,1099


## References ###
[1] https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059